[**Connect 4**](https://www.kaggle.com/competitions/connect-4)

[**Connect X**](https://www.kaggle.com/competitions/connectx/leaderboard)


[debugger](https://www.kaggle.com/vyacheslavbolotin/debugger-c4)

[round_robin](https://www.kaggle.com/code/dott1718/kdb-workshop)

[agent_ConnectX - Minimax_20x_faster](https://www.kaggle.com/code/sidagar/making-minimax-20x-faster) - [*Sidhant Agarval*](https://www.kaggle.com/sidagar)

[agent_ConnectX - 5x_Faster_Heuristic](https://www.kaggle.com/code/sorkun/5x-faster-heuristic) - [*Murat Cihan Sorkun*](https://www.kaggle.com/code/sorkun/5x-faster-heuristic)

[agent_ConnectX - Fastest N-step Lookahead Minimax with CYTHON](https://www.kaggle.com/code/jwilliamhughdore/fastest-n-step-lookahead-minimax-agent-with-cython)

[agent_ConnectX_Kaggle_Learn_*AlexisCook*](https://www.kaggle.com/code/alexisbcook/n-step-lookahead) + [Alpha–beta pruning](https://en.wikipedia.org/wiki/Alpha%E2%80%93beta_pruning) + 
[Faster Numba](https://www.kaggle.com/code/garyongguanjie/numba-speedtest)

In [1]:
import os
import time
import copy
import random
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from IPython.display import clear_output
from kaggle_environments import make, evaluate

No pygame installed, ignoring import


## [Minimax_20x_faster Sidhant Agarval](https://www.kaggle.com/code/sidagar/making-minimax-20x-faster)

In [2]:
def ag_minimax_20xfaster(obs, config):
    
    import random
    import numpy as np

    def score_move_a(grid, col, mark, config, start_score, n_steps):
        next_grid, pos = drop_piece(grid, col, mark, config)
        row, col = pos
        score = get_heuristic_optimised(grid,next_grid,mark,config, row, col,start_score)
        valid_moves = [col for col in range (config.columns) if next_grid[0][col]==0]
        #Since we have just dropped our piece there is only the possibility of us getting 4 in a row and not the opponent.
        #Thus score can only be +infinity.
        scores = []
        if len(valid_moves)==0 or n_steps ==0 or score == float("inf"):
            return score
        else :
            for col in valid_moves:
                current = score_move_b(next_grid,col,mark,config,score,n_steps-1)
                scores.append(current)
            score = min(scores)
        return score


    def score_move_b(grid, col, mark, config, start_score, n_steps):
        next_grid, pos = drop_piece(grid,col,(mark%2)+1,config)
        row, col = pos
        score = get_heuristic_optimised(grid,next_grid,mark,config, row, col,start_score)
        valid_moves = [col for col in range (config.columns) if next_grid[0][col]==0]

        #The converse is true here.
        #Since we have just dropped opponent piece there is only the possibility of opponent getting 4 in a row and not us.
        #Thus score can only be -infinity.
        scores = []
        if len(valid_moves)==0 or n_steps ==0 or score == float ("-inf"):
            return score
        else :
            for col in valid_moves:
                current = score_move_a (next_grid,col,mark,config,score,n_steps-1)
                scores.append(current)
            score = max(scores)
        return score


    def drop_piece(grid, col, mark, config):
        next_grid = grid.copy()
        for row in range(config.rows-1, -1, -1):
            if next_grid[row][col] == 0:
                break
        next_grid[row][col] = mark
        return next_grid,(row,col)


    def get_heuristic(grid, mark, config):
        score = 0
        num = count_windows(grid,mark,config)
        for i in range(config.inarow):
            #num  = count_windows (grid,i+1,mark,config)
            if (i==(config.inarow-1) and num[i+1] >= 1):
                return float("inf")
            score += (4**(i))*num[i+1]
        num_opp = count_windows (grid,mark%2+1,config)
        for i in range(config.inarow):
            if (i==(config.inarow-1) and num_opp[i+1] >= 1):
                return float ("-inf")
            score-= (2**((2*i)+1))*num_opp[i+1]
        return score


    def get_heuristic_optimised(grid, next_grid, mark, config, row, col, start_score):
        score = 0
        num1 = count_windows_optimised(grid,mark,config,row,col)
        num2 = count_windows_optimised(next_grid,mark,config,row,col)
        for i in range(config.inarow):
            if (i==(config.inarow-1) and (num2[i+1]-num1[i+1]) >= 1):
                return float("inf")
            score += (4**(i))*(num2[i+1]-num1[i+1])
        num1_opp = count_windows_optimised(grid,mark%2+1,config,row,col)
        num2_opp = count_windows_optimised(next_grid,mark%2+1,config,row,col)
        for i in range(config.inarow): 
            if (i==(config.inarow-1) and num2_opp[i+1]-num1_opp[i+1]  >= 1):
                return float ("-inf")     
            score-= (2**((2*i)+1))*(num2_opp[i+1]-num1_opp[i+1])
        score+= start_score
        return score


    def check_window(window, piece, config):
        if window.count((piece%2)+1)==0:
            return window.count(piece)
        else: return -1
    
    
    def count_windows(grid, piece, config):
        num_windows = np.zeros(config.inarow+1)
        # horizontal
        for row in range(config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[row, col:col+config.inarow])
                type_window = check_window(window, piece, config)
                if type_window != -1:
                    num_windows[type_window] += 1
        # vertical
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns):
                window = list(grid[row:row+config.inarow, col])
                type_window = check_window(window, piece, config)
                if type_window != -1:
                    num_windows[type_window] += 1
        # positive diagonal
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[range(row, row+config.inarow), range(col, col+config.inarow)])
                type_window = check_window(window, piece, config)
                if type_window != -1:
                    num_windows[type_window] += 1
        # negative diagonal
        for row in range(config.inarow-1, config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
                type_window = check_window(window, piece, config)
                if type_window != -1:
                    num_windows[type_window] += 1
        return num_windows
    
        
    def count_windows_optimised(grid, piece, config, row, col):
        num_windows = np.zeros(config.inarow+1)
        # horizontal
        for acol in range(max(0,col-(config.inarow-1)),min(col+1,(config.columns-(config.inarow-1)))):
            window = list(grid[row, acol:acol+config.inarow])
            type_window = check_window(window, piece, config)
            if type_window != -1:
                num_windows[type_window] += 1
        # vertical
        for arow in range(max(0,row-(config.inarow-1)),min(row+1,(config.rows-(config.inarow-1)))):
            window = list(grid[arow:arow+config.inarow, col])
            type_window = check_window(window, piece, config)
            if type_window != -1:
                num_windows[type_window] += 1
        # positive diagonal
        for arow, acol in zip(range(row-(config.inarow-1),row+1),range(col-(config.inarow-1),col+1)):
            if (arow>=0 and acol>=0 and arow<=(config.rows-config.inarow) and acol<=(config.columns-config.inarow)):
                window = list(grid[range(arow, arow+config.inarow), range(acol, acol+config.inarow)])
                type_window = check_window(window, piece, config)
                if type_window != -1:
                    num_windows[type_window] += 1
        # negative diagonal
        for arow,acol in zip(range(row,row+config.inarow),range(col,col-config.inarow,-1)):
            if (arow >= (config.inarow-1) and acol >=0 and arow <= (config.rows-1) and acol <= (config.columns-config.inarow)):
                window = list(grid[range(arow, arow-config.inarow, -1), range(acol, acol+config.inarow)])
                type_window = check_window(window, piece, config)
                if type_window != -1:
                    num_windows[type_window] += 1
        return num_windows
    
    N_STEP = 4
    
    s = np.sum(np.array(obs.board))

    if s == 1: return random.choice([0, 1, 2,2, 3,3,3, 4,4, 5])
    if s <= 4: return 3
    if s <= 7:
        if obs.board[36]==0: return 1
        if obs.board[40]==0: return 5

    grid = np.asarray(obs.board).reshape(config.rows, config.columns)
    valid_moves = [c for c in range(config.columns) if grid[0][c] == 0]
    scores = {}
    start_score = get_heuristic(grid, obs.mark, config)
    for col in valid_moves:
        scores[col] = score_move_a(grid, col, obs.mark, config, start_score, N_STEP)
    # print ("4 Step lookahead agent:",scores)
    max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]
    return random.choice(max_cols)

## [5x_Faster_Heuristic Murat Cihan Sorkun](https://www.kaggle.com/code/sorkun/5x-faster-heuristic)

In [3]:
def ag_heuristic5xfaster(obs, config):
    import numpy as np
    import random

    ROWS    = config.rows
    COLUMNS = config.columns
    CNCTX   = config.inarow
   
    if obs.board.count(0) >= int(0.35 * ROWS * COLUMNS):
        N_STEPS = 5
    else:
        N_STEPS = 6
         
    # Gets board at next step if agent drops piece in selected column
    def drop_piece(grid, col, mark):
        next_grid = grid.copy()
        for row in range(ROWS-1, -1, -1):
            if next_grid[row][col] == 0:
                break
        next_grid[row][col] = mark
        return next_grid
    
    
    def get_fast_heuristic(grid, mark, config):
        # Convert the grid into a string (default + 90deg rotated + all possible diagonals)     
        horizontal_str=str(grid)
        vertical_str=str(grid.transpose())
        diagonal_str=""
        
        for col in range(0,4):
            for row in range(0,3):    
                for i in range(4):
                    diagonal_str+=str(grid[row+i][col+i])+" "
                diagonal_str+=","
            for row in range(3,6):    
                for i in range(4):
                    diagonal_str+=str(grid[row-i][col+i])+" "
                diagonal_str+=","

        # Combine all strings into a single one         
        combined_str= horizontal_str + vertical_str + diagonal_str
        # print(combined_str)

        str_piece=str(mark)
        search_str4=str_piece+" "+str_piece+" "+str_piece+" "+str_piece    
        num_fours=combined_str.count(search_str4)
        # print(num_fours)

        # All possible three cases    
        search_str3_1=str_piece+" "+str_piece+" "+str_piece+" 0"    
        search_str3_2=str_piece+" "+str_piece+" 0 "+str_piece 
        search_str3_3=str_piece+" 0 "+str_piece+" "+str_piece 
        search_str3_4="0 "+str_piece+" "+str_piece+" "+str_piece

        num_threes = combined_str.count(search_str3_1)+combined_str.count(search_str3_2)+combined_str.count(search_str3_3)+combined_str.count(search_str3_4)
        # print(num_threes)

        # All possible two cases   
        search_str2_1=str_piece+" "+str_piece+" 0 0"    
        search_str2_2=str_piece+" 0 0 "+str_piece 
        search_str2_3="0 0 "+str_piece+" "+str_piece 
        search_str2_4="0 "+str_piece+" 0 "+str_piece
        search_str2_5=str_piece+" 0 "+str_piece+" 0"
        search_str2_6="0 "+str_piece+" "+str_piece+" 0"

        num_twos = combined_str.count(search_str2_1)+combined_str.count(search_str2_2)+combined_str.count(search_str2_3)+combined_str.count(search_str2_4)+combined_str.count(search_str2_5)+combined_str.count(search_str2_6)
        # print(num_twos)

        # All possible three cases  (opp)
        str_piece=str(mark%2+1)
        search_str3_1=str_piece+" "+str_piece+" "+str_piece+" 0"    
        search_str3_2=str_piece+" "+str_piece+" 0 "+str_piece 
        search_str3_3=str_piece+" 0 "+str_piece+" "+str_piece 
        search_str3_4="0 "+str_piece+" "+str_piece+" "+str_piece

        num_threes_opp = combined_str.count(search_str3_1)+combined_str.count(search_str3_2)+combined_str.count(search_str3_3)+combined_str.count(search_str3_4)
        # print(num_threes_opp)

        #All possible two cases  (opp)  
        search_str2_1=str_piece+" "+str_piece+" 0 0"    
        search_str2_2=str_piece+" 0 0 "+str_piece 
        search_str2_3="0 0 "+str_piece+" "+str_piece 
        search_str2_4="0 "+str_piece+" 0 "+str_piece
        search_str2_5=str_piece+" 0 "+str_piece+" 0"
        search_str2_6="0 "+str_piece+" "+str_piece+" 0"

        num_twos_opp = combined_str.count(search_str2_1)+combined_str.count(search_str2_2)+combined_str.count(search_str2_3)+combined_str.count(search_str2_4)+combined_str.count(search_str2_5)+combined_str.count(search_str2_6)
        # print(num_twos_opp)

        # Calculate score 
        score =  1e2*num_twos - 2e2*num_twos_opp  + 1e4*num_threes - 1e5*num_threes_opp + 1e8*num_fours
        return score

    # Helper function for minimax: checks if agent or opponent has four in a row in the window
    def is_terminal_window(window):
        return window.count(1) == CNCTX or window.count(2) == CNCTX

    # Helper function for minimax: checks if game has ende
    def is_terminal_node(grid):
        # Check for draw 
        if list(grid[0, :]).count(0) == 0:
            return True
        # Check for win: horizontal, vertical, or diagonal
        # horizontal 
        for row in range(ROWS):
            for col in range(COLUMNS-(CNCTX-1)):
                window = list(grid[row, col:col+CNCTX])
                if is_terminal_window(window):
                    return True
        # vertical
        for row in range(ROWS-(CNCTX-1)):
            for col in range(COLUMNS):
                window = list(grid[row:row+CNCTX, col])
                if is_terminal_window(window):
                    return True
        # positive diagonal
        for row in range(ROWS-(CNCTX-1)):
            for col in range(COLUMNS-(CNCTX-1)):
                window = list(grid[range(row, row+CNCTX), range(col, col+CNCTX)])
                if is_terminal_window(window):
                    return True
        # negative diagonal
        for row in range(CNCTX-1, ROWS):
            for col in range(COLUMNS-(CNCTX-1)):
                window = list(grid[range(row, row-CNCTX, -1), range(col, col+CNCTX)])
                if is_terminal_window(window):
                    return True
        return False

    # Minimax implementation was here:
    def minimax_ab(node, depth, alpha, beta, maximizingPlayer, mark, config):

        if depth == 0 or is_terminal_node(node):
            return get_fast_heuristic(node, mark, config) # get_heuristic(node, mark)
        
        valid_moves = [c for c in range(config.columns) if node[0][c] == 0]

        if maximizingPlayer:
            value = -np.Inf
            for col in valid_moves:
                child = drop_piece(node, col, mark)
                value = max(value, minimax_ab(child, depth-1, alpha, beta, False, mark, config))
                alpha = max(alpha, value)
                if alpha >= beta:
                    break
            return value
        else:
            value = np.Inf
            for col in valid_moves:
                child = drop_piece(node, col, mark%2+1)
                value = min(value, minimax_ab(child, depth-1, alpha, beta, True, mark, config))
                beta  = min(beta,value)
                if beta <= alpha:
                    break 
            return value
  
    def score_move(grid, col, mark, config, nsteps):
        next_grid = drop_piece(grid, col, mark)
        score  = minimax_ab(next_grid, nsteps-1, -np.Inf, np.Inf, False, mark, config)
        return score
    
    
    s = np.sum(np.array(obs.board))
    
    if s == 1: return random.choice([0, 1, 2,2, 3,3,3, 4,4, 5])
    if s <= 4: return 3
    # if s <= 7:
    #     if obs.board[36]==0: return 1
    #     if obs.board[40]==0: return 5

    valid_moves = [c for c in range(COLUMNS) if obs.board[c] == 0]

    grid = np.asarray(obs.board).reshape(ROWS, COLUMNS)

    # Use the heuristic to assign a score to each possible board in the next step 
    scores = dict(zip(valid_moves, [score_move(grid, col, obs.mark, config, N_STEPS) for col in valid_moves]))

    # Get a list of columns (moves) that maximize the heuristic
    max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]

    # Select at random from the maximizing columns
    return random.choice(max_cols)

## [Fastest N-step Lookahead Minimax with CYTHON](https://www.kaggle.com/code/jwilliamhughdore/fastest-n-step-lookahead-minimax-agent-with-cython) - [J. William Hugh Dore](https://www.kaggle.com/jwilliamhughdore)

In [4]:
%load_ext Cython

In [5]:
%%cython 
from cython.view cimport array as cvarray

cimport numpy as np
import numpy as np
import pickle 
import random

DTYPE = np.intc 


cpdef  np.ndarray[dtype= int , ndim=2] drop_piece( np.ndarray[ dtype=np.int32_t,ndim=2] grid,int col,int mark):
    ''' Gets board at next step if agent drops piece in selected column
    Purposefully creates a new board image so as not to corrupt the old one, useful behavior as same grid needs to be pass into a different
    possbile move in order for evaluation'''
    cdef np.ndarray[dtype= int , ndim=2] next_grid= grid.copy()
    cdef int [:,:] nxtgridview = next_grid
    for row in range(5, -1, -1):
        if nxtgridview[row][col] == 0:
            break
    nxtgridview[row][col] = mark
    return next_grid

# # Helper function for get_heuristic: checks if window satisfies heuristic conditions
cdef int count( int[4] window,int piece):
    cdef int count=0
    for i in range (4):
#         if window[i] != 0 or window[i] != 1 or window[i] != 2 :
#             filehandler = open('ERROR IN WINDOW ', 'wb') #######################################################################################
#             pickle.dump(count, filehandler)##########################################################               
        if window[i]== piece:
            count += 1
    return count    

cdef bint check_window( int[4] window,int num_discs,int piece ):
    return ( (count(window, piece) == num_discs) and (count(window,0) == 4-num_discs) )

# Helper function for get_heuristic: counts number of windows satisfying specified heuristic conditions

cdef int count_windows(np.ndarray[ dtype=np.int32_t,ndim=2] grid,int num_discs,int piece):
    cdef int num_windows = 0
    cdef int[4] window
    cdef int i =0
    # horizontal
    for row in range(6):
        for col in range(4):
            window = (grid[row, col:col+4]) 
            if check_window(window, num_discs, piece):
                num_windows += 1
    # vertical
    for row in range(3):
        for col in range(7):
            window = (grid[row:row+4, col]) 
            if check_window(window, num_discs, piece):
                num_windows += 1
    # positive diagonal
    for row in range(3):
        i=0
        for col in range(4):
            for i in range (4):
                window[i] = (grid[row+i,col+i])
            if check_window(window, num_discs, piece):
                num_windows += 1
    # negative diagonal
    for row in range(3, 6):
        i=0
        for col in range(4):
            for i in range (4):
                window[i] = (grid[row-i, col+i])
            if check_window(window, num_discs, piece):
                num_windows += 1
    return num_windows

    
# Helper function for minimax: calculates value of heuristic for grid
cdef int get_heuristic( np.ndarray[ dtype=np.int32_t,ndim=2] grid, int mark):
    cdef int score=0
    score = count_windows(grid, 3, mark) \
            - 1*count_windows(grid, 3, mark%2+1) \
            - 10000*count_windows(grid, 4, mark%2+1 ) \
            + 1000000*count_windows(grid, 4, mark)
    return score

# Uses minimax to calculate value of dropping piece in selected column
cdef int score_move(np.ndarray[ dtype=np.int32_t,ndim=2]grid, int col,int mark, int  nsteps):
    cdef np.ndarray[ dtype=np.int32_t,ndim=2] next_grid = drop_piece(grid, col, mark)
    score = minimax(next_grid, nsteps-1, False, mark)    
    
    return score

# Helper function for minimax: checks if agent or opponent has four in a row in the window
cdef bint is_terminal_window(int[4] window):
    return ( (count(window,1) == 4)or (count(window,2) == 4))

# Helper function for minimax: checks if game has ended
cdef bint is_terminal_node(np.ndarray[ dtype=np.int32_t,ndim=2] grid) :
    cdef int num_windows = 0
    cdef int [4] window
    cdef bint not_draw = False
    # Check for draw 
    for i in range(7):
        if grid [0, i] == 0:
            not_draw = True
    if not_draw == False:
        return True
    # Check for win: horizontal, vertical, or diagonal
    # horizontal
    for row in range(6):
        for col in range(4):
            window = (grid[row, col:col+4])
            if is_terminal_window(window):
                    return True
    # vertical
    for row in range(3):
        for col in range(7):
            window = (grid[row:row+4, col])
            if is_terminal_window(window):
                    return True
    # positive diagonal
    for row in range(3):
        for col in range(4):
#             window = (grid[range(row, row+4), range(col, col+4)])
            for i in range (4):
                window[i] = (grid[row+i,col+i])
            if is_terminal_window(window):
                return True

    # negative diagonal
    for row in range(3, 6):
        for col in range(4):
#             window = (grid[range(row, row-4, -1), range(col, col+4)])
            for i in range (4):
                window[i] = (grid[row-i, col+i])
            if is_terminal_window(window):
                return True
    return False



# Minimax implementation with alfabeta pruning
cdef int minimax(np.ndarray[ dtype=np.int32_t,ndim=2] Pnode,int depth,int maximizingPlayer,int mark):
    cdef bint is_terminal
    cdef int value
    cdef int maxv
    cdef int miniv
    cdef np.ndarray[ dtype=np.int32_t,ndim=2] node = Pnode.copy()
    cdef int [:,:] node_view = node
    cdef int col
    cdef np.ndarray[dtype= int , ndim=2] child 
    cdef int [7] val_m#valid 
    cdef int m_c=0#valid moves count
    # Create valid moves list
    for c in range(7):
        if node_view [0][c]==0 :
            val_m[m_c]=c
            m_c += 1
        
    is_terminal = is_terminal_node(node)
    if is_terminal:
        value =get_heuristic(node, mark) # сокращение альфабета, кодирующее глубину выигрышного пути в счете. Так как не
        if value > 800000:               # потрудитесь искать похожие или более длинные пути.
              value= 800000+ 10*(depth)  # alfabeta pruning encoding the depth of a winning path into the score. So as not 
        return value                     # bother looking for similar or longer paths.
    if depth == 0 :
        value=get_heuristic(node, mark)
        return value

    if maximizingPlayer:
        value = - 200000000 
        for c in range(m_c):
            col = val_m[c] 
            child = drop_piece(node,col, mark)  
            maxv = minimax(child, depth-1, False, mark)             
            value = max(value, maxv)
            if value >= 800000 :##ALFA BETA PRUNING
                winning_depth = (value -800000)%10
                if winning_depth >= (depth-1):
                    break  #i  Could only come up with an equally fast path. remeber depth goes down. 
        return value       #i Оставалось только придумать столь же быстрый путь. помните, что глубина уменьшается.
    else:
        value =  200000000 
        for c in range(m_c):
            col = val_m[c]
            child = drop_piece(node,col,mark%2+1)
            miniv = minimax(child, depth-1, True, mark)
            value = min(value,miniv )
            if value < -8000:##ALFA BETA PRUNING
                break   
        return value

cpdef int ag__faster__CYTHON__(obs, config): 

    cdef np.ndarray[dtype= np.int32_t , ndim=2] grid = np.array(obs.board,np.intc ).reshape(6, 7)
    cdef np.ndarray[ dtype=np.int32_t,ndim=1] valid_moves  
    cdef int [7] val_m#valid 
    cdef int m_c=0#valid moves count
    cdef int counter=1 
    cdef int [:,:] gridview = grid    
    # Create valid moves list
    for c in range(7):
        if gridview [0][c]==0 :
            val_m[m_c]=c
            m_c += 1

    # Use the heuristic to assign a score to each possible board in the next step
    
    if m_c == 7: N_STEPS=4
    if m_c == 6: N_STEPS=5
    if m_c == 5: N_STEPS=6
    if m_c == 4: N_STEPS=7
    if m_c == 3: N_STEPS=8
    if m_c == 2: N_STEPS=9
    if m_c == 1: N_STEPS=10
        
    # if m_c == 7 and (obs.board.count(0) < int(0.35 * config.rows * config.columns)): N_STEPS=5
        
    scores = dict(zip(val_m,[score_move(grid, val_m[i], obs.mark, N_STEPS) for i in range(m_c) ] ))
    # Get a list of columns (moves) that maximize the heuristic
    max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]
    return max_cols[len(max_cols)//2]

Content of stderr:
In file included from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarraytypes.h:1940,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from /root/.cache/ipython/cython/_cython_magic_185a11676e9661feae08d48d7f9cac264fb0347f.c:1107:
/opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~

## [Kaggle_Learn Alexis Cook](https://www.kaggle.com/code/alexisbcook/n-step-lookahead) + [Alpha–beta pruning](https://en.wikipedia.org/wiki/Alpha%E2%80%93beta_pruning) + [Faster Numba](https://www.kaggle.com/code/garyongguanjie/numba-speedtest)

In [6]:
from numba import jit

def np_count_0(window): 
    return len(window) - np.count_nonzero(window)

In [7]:
@jit
def jnpc0(window): 
    return len(window) - np.count_nonzero(window)

/tmp/ipykernel_18/3510957661.py:2: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def jnpc0(window):


In [8]:
%%timeit
np_count_0(np.asarray([0,2,1,0]))

3.58 µs ± 35.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [9]:
jnpc0(np.asarray([0,2,1,0]))

2

In [10]:
%%timeit
jnpc0(np.asarray([0,2,1,0]))

2.18 µs ± 21.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [11]:
def ag_KLAlexisCook_abnj(obs, config):
    import random
    import numpy as np

    # Gets board at next step if agent drops piece in selected column
    def drop_piece(grid, col, mark, config):
        next_grid = grid.copy()
        for row in range(config.rows-1, -1, -1):
            if next_grid[row][col] == 0:
                break
        next_grid[row][col] = mark
        return next_grid

#     # Helper function for get_heuristic: checks if window satisfies heuristic conditions
#     def check_window(window, num_discs, piece, config):
#         return (window.count(piece) == num_discs and window.count(0) == config.inarow-num_discs)
    
#     # Helper function for get_heuristic: counts number of windows satisfying specified heuristic conditions
#     def count_windows(grid, num_discs, piece, config):
#         num_windows = 0
#         # horizontal
#         for row in range(config.rows):
#             for col in range(config.columns-(config.inarow-1)):
#                 window = list(grid[row, col:col+config.inarow])
#                 if check_window(window, num_discs, piece, config):
#                     num_windows += 1
#         # vertical
#         for row in range(config.rows-(config.inarow-1)):
#             for col in range(config.columns):
#                 window = list(grid[row:row+config.inarow, col])
#                 if check_window(window, num_discs, piece, config):
#                     num_windows += 1
#         # positive diagonal
#         for row in range(config.rows-(config.inarow-1)):
#             for col in range(config.columns-(config.inarow-1)):
#                 window = list(grid[range(row, row+config.inarow), range(col, col+config.inarow)])
#                 if check_window(window, num_discs, piece, config):
#                     num_windows += 1
#         # negative diagonal
#         for row in range(config.inarow-1, config.rows):
#             for col in range(config.columns-(config.inarow-1)):
#                 window = list(grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
#                 if check_window(window, num_discs, piece, config):
#                     num_windows += 1
#         return num_windows
    
    # Helper function for get_heuristic: counts number of windows satisfying specified heuristic conditions
    def count_windows(grid, num_discs, piece, config):
        num_windows = 0
        # horizontal
        for row in range(config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = grid[row, col:col+config.inarow]
                if np.count_nonzero(window==piece) == num_discs and jnpc0(window) == config.inarow-num_discs:
                    num_windows += 1
        # vertical
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns):
                window = grid[row:row+config.inarow, col]
                if np.count_nonzero(window==piece) == num_discs and jnpc0(window) == config.inarow-num_discs:
                    num_windows += 1
        # positive diagonal
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns-(config.inarow-1)):
                window = grid[range(row, row+config.inarow), range(col, col+config.inarow)]
                if np.count_nonzero(window==piece) == num_discs and jnpc0(window) == config.inarow-num_discs:
                    num_windows += 1
        # negative diagonal
        for row in range(config.inarow-1, config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)]
                if np.count_nonzero(window==piece) == num_discs and jnpc0(window) == config.inarow-num_discs:
                    num_windows += 1
        return num_windows

    # Helper function for minimax: calculates value of heuristic for grid
    def get_heuristic(grid, mark, config):
        num_threes = count_windows(grid, 3, mark, config)
        num_fours = count_windows(grid, 4, mark, config)
        num_threes_opp = count_windows(grid, 3, mark%2+1, config)
        num_fours_opp = count_windows(grid, 4, mark%2+1, config)
        score = num_threes - 1e2*num_threes_opp - 1e4*num_fours_opp + 1e6*num_fours
        return score
    
    # Uses minimax to calculate value of dropping piece in selected column
    def score_move(grid, col, mark, config, nsteps):
        next_grid = drop_piece(grid, col, mark, config)
        # score = minimax(next_grid, nsteps-1, False, mark, config)
        score = minimax_ab(next_grid, nsteps-1, -np.Inf, np.Inf, False, mark, config)
        return score

#     # Helper function for minimax: checks if agent or opponent has four in a row in the window
#     def is_terminal_window(window, config):
#         return window.count(1) == config.inarow or window.count(2) == config.inarow

#     # Helper function for minimax: checks if game has ended
#     def is_terminal_node(grid, config):
#         # Check for draw 
#         if list(grid[0, :]).count(0) == 0:
#             return True
#         # Check for win: horizontal, vertical, or diagonal
#         # horizontal 
#         for row in range(config.rows):
#             for col in range(config.columns-(config.inarow-1)):
#                 window = list(grid[row, col:col+config.inarow])
#                 if is_terminal_window(window, config):
#                     return True
#         # vertical
#         for row in range(config.rows-(config.inarow-1)):
#             for col in range(config.columns):
#                 window = list(grid[row:row+config.inarow, col])
#                 if is_terminal_window(window, config):
#                     return True
#         # positive diagonal
#         for row in range(config.rows-(config.inarow-1)):
#             for col in range(config.columns-(config.inarow-1)):
#                 window = list(grid[range(row, row+config.inarow), range(col, col+config.inarow)])
#                 if is_terminal_window(window, config):
#                     return True
#         # negative diagonal
#         for row in range(config.inarow-1, config.rows):
#             for col in range(config.columns-(config.inarow-1)):
#                 window = list(grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
#                 if is_terminal_window(window, config):
#                     return True
#         return False
    
    def is_terminal_node(grid, config):
        # Check for draw 
        top_row = grid[0, :]
        if len(top_row) - np.count_nonzero(top_row) == 0:
            return True
        # Check for win: horizontal, vertical, or diagonal
        # horizontal 
        for row in range(config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = grid[row, col:col+config.inarow]
                if np.count_nonzero(window==1) == config.inarow or np.count_nonzero(window==2) == config.inarow:
                    return True
        # vertical
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns):
                window = grid[row:row+config.inarow, col]
                if np.count_nonzero(window==1) == config.inarow or np.count_nonzero(window==2) == config.inarow:
                    return True
        # positive diagonal
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns-(config.inarow-1)):
                window = grid[range(row, row+config.inarow), range(col, col+config.inarow)]
                if np.count_nonzero(window==1) == config.inarow or np.count_nonzero(window==2) == config.inarow:
                    return True
        # negative diagonal
        for row in range(config.inarow-1, config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)]
                if np.count_nonzero(window==1) == config.inarow or np.count_nonzero(window==2) == config.inarow:
                    return True
        return False

#     # Minimax implementation
#     def minimax(node, depth, maximizingPlayer, mark, config):
#         is_terminal = is_terminal_node(node, config)
#         valid_moves = [c for c in range(config.columns) if node[0][c] == 0]
#         if depth == 0 or is_terminal:
#             return get_heuristic(node, mark, config)
#         if maximizingPlayer:
#             value = -np.Inf
#             for col in valid_moves:
#                 child = drop_piece(node, col, mark, config)
#                 value = max(value, minimax(child, depth-1, False, mark, config))
#             return value
#         else:
#             value = np.Inf
#             for col in valid_moves:
#                 child = drop_piece(node, col, mark%2+1, config)
#                 value = min(value, minimax(child, depth-1, True, mark, config))
#             return value
        
    # Minimax implementation, with alpha/beta-pruning
    def minimax_ab(node, depth, alpha, beta, maximizingPlayer, mark, config):

        if depth == 0 or is_terminal_node(node, config):
            return get_heuristic(node, mark, config)
        
        valid_moves = [c for c in range(config.columns) if node[0][c] == 0]

        if maximizingPlayer:
            value = -np.Inf
            for col in valid_moves:
                child = drop_piece(node, col, mark, config)
                value = max(value, minimax_ab(child, depth-1, alpha, beta, False, mark, config))
                alpha = max(alpha, value)
                if alpha >= beta:
                    break
            return value
        else:
            value = np.Inf
            for col in valid_moves:
                child = drop_piece(node, col, mark%2+1, config)
                value = min(value, minimax_ab(child, depth-1, alpha, beta, True, mark, config))
                beta  = min(beta,value)
                if beta <= alpha:
                    break 
            return value
        
    # begin to work..
    
    # How deep to make the game tree: higher values take longer to run!
    
    N_STEPS = 4 # N_STEPS__Kaggle_Learn_AlexisCook_abn 
    
    # Get list of valid moves
    valid_moves = [c for c in range(config.columns) if obs.board[c] == 0]
    
    if len(valid_moves) == 6 : N_STEPS = 5
    if len(valid_moves) == 5 : N_STEPS = 6
    if len(valid_moves) == 4 : N_STEPS = 7
    if len(valid_moves) == 3 : N_STEPS = 8
                
    # Convert the board to a 2D grid
    grid = np.asarray(obs.board).reshape(config.rows, config.columns)
    # Use the heuristic to assign a score to each possible board in the next step
    scores = dict(zip(valid_moves, [score_move(grid, col, obs.mark, config, N_STEPS) for col in valid_moves]))
    # Get a list of columns (moves) that maximize the heuristic
    max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]
    # Select at random from the maximizing columns
    return random.choice(max_cols)

## [debugger](https://www.kaggle.com/vyacheslavbolotin/debugger-c4)

In [12]:
def debugger_c4(list_of_agents_to_trace):
    map4q = '''
         01,02,03,04; 31,32,33,34;                 32,33,34,35; 33,34,35,36; 
           40,41,42,43; 41,42,43,44;             42,43,44,45; 43,44,45,46; 
             50,51,52,53; 51,52,53,54;         52,53,54,55; 30,31,32,33;
               04,14,24,34; 14,24,34,44;     24,34,44,54; 05,15,25,35; 
                 06,16,26,36; 03,12,21,30; 15,25,35,45; 25,35,45,55; 
                   16,26,36,46; 26,36,46,56;          23,34,45,56;
                     04,13,22,31; 13,22,31,40;      03,14,25,36; 
                       05,14,23,32; 14,23,32,41;  20,30,40,50; 
                         06,15,24,33; 15,24,33,42;
                           24,33,42,51; 23,32,41,50;
                             03,13,23,33; 13,23,33,43; 
                               16,25,34,43; 25,34,43,52; 
                                 11,22,33,44; 11,21,31,41;
                     01,12,23,34;  02,13,24,35; 12,23,34,45; 
                   00,11,22,33;      26,35,44,53; 23,33,43,53;              
                 02,12,22,32;          21,32,43,54; 22,33,44,55; 
               00,10,20,30; 22,23,24,25; 21,31,41,51; 13,24,35,46;
             01,11,21,31; 10,20,30,40;     12,22,32,42; 22,32,42,52;
           00,01,02,03; 10,21,32,43;         02,03,04,05; 03,04,05,06;
         10,11,12,13; 11,12,13,14;             12,13,14,15; 13,14,15,16;
       20,21,22,23; 21,22,23,24;                 23,24,25,26; 20,31,42,53; 53,54,55,56'''
    
    # -----------------------------------------------------------------------------------------------
    start_games_at_away_first_agent = False # True # => The first agent from the list will start his game either “from home” or “away”,
    only_observation_first_agent    = False # True # => Everyone will play two games with the first: one “at home”, the other “away”, or everyone will play with everyone
    # -----------------------------------------------------------------------------------------------
    class Obs:
        def __init__(self, board, mark):
            self.board = board
            self.mark = mark
    # -----------------------------------------------------------------------------------------------
    class Config:
        def __init__(self, rows, columns, inarow):
            self.rows = rows
            self.columns = columns
            self.inarow = inarow
    # -----------------------------------------------------------------------------------------------
    import time
    import numpy as np
    # -----------------------------------------------------------------------------------------------
    Rows, Cols = 6, 7
    config = Config(Rows, Cols, 4)
    AT2 = (np.array([a for a in range(42)])).reshape(Rows, Cols)
    # -----------------------------------------------------------------------------------------------
    def yx(s): return AT2[int(s[0])][int(s[1])]
    Ss = [x.split(",") for x in [x for x in map4q.replace(" ", "").replace("\n", "").split(";")]]
    iMap4q = [list(map(yx, s)) for s in Ss]
    # -----------------------------------------------------------------------------------------------
    def gyx(s): return [int(s[0]), int(s[1])]
    gSs = [x.split(",") for x in [x for x in map4q.replace(" ", "").replace("\n", "").split(";")]]
    gMap4q = [list(map(gyx, s)) for s in gSs]
    # -----------------------------------------------------------------------------------------------
    def inject(col, mark, board):
        i_krai = config.columns * (config.rows - 1) + col
        for i in range(i_krai, -1, -7):
            if board[i] == 0:
                new_board = board.copy()
                new_board[i] = mark
                return new_board
        return board
    # -----------------------------------------------------------------------------------------------
    def win_4o_in(board):
        grid = np.asarray(board).reshape(6,7)
        for i in gMap4q:
            v = grid[i[0][0],i[0][1]] * grid[i[1][0],i[1][1]] * grid[i[2][0],i[2][1]] * grid[i[3][0],i[3][1]]
            if v == 1 or v == 16: return True
        return False
    # -----------------------------------------------------------------------------------------------
    def tsum(reagent):
        home = sum(reagent["games"][0]) if len(reagent["games"][0])>0 else 0
        away = sum(reagent["games"][1]) if len(reagent["games"][1])>0 else 0
        return home + away
    # -----------------------------------------------------------------------------------------------
    def ttime(reagent, gt):
        time = reagent["time"][0]  if len(reagent["time"])>0 else 0
        return [round(time+gt, 2)]
    # -----------------------------------------------------------------------------------------------
    def tmoves(reagent, ms):
        moves = reagent["moves"][0] if len(reagent["moves"])>0 else 0
        return [moves+ms]
    # -----------------------------------------------------------------------------------------------
    def tper1move(reagent):
        time = reagent["time"][0]   if len(reagent["time"]) > 0 else 0
        moves = reagent["moves"][0] if len(reagent["moves"])> 0 else 0
        return [round(time / moves, 1)]
    # -----------------------------------------------------------------------------------------------
    def print_to_clip(board, ims, i, attack, defense, stime1, stime2, s1, s2):
        print("1.{0} [{1}]   2.{2} [{3}]".format(name(attack), s1, name(defense), s2),
              "  1.Time =", round(stime1, 2), '  2.Time =', round(stime2, 2), '  q.moves:', i,
              "  1.speed =", round(stime1 / i, 1), "  2.speed =", round(stime2 / i, 1),
              "  \n\nGame:", str(ims).replace(", ", ","),
              "  \n\nBoard:", str(board).replace(", ", ","),"\n")
        bn2d = np.array(board).reshape(Rows, Cols)
        for r in range(len(bn2d)): print(bn2d[r])
        print('{0}{1}'.format('\n',"="*101))
    # -----------------------------------------------------------------------------------------------
    def rec(total_points, _A, _D, ia, id, stime1, stime2):
        reatta = total_points[name(Attack)]
        redefe = total_points[name(Defense)]
        reatta["games"][0].extend([_A])
        redefe["games"][1].extend([_D])
        reatta["T"] = [tsum(reatta)]
        redefe["T"] = [tsum(redefe)]
        reatta["time"] = ttime(reatta, stime1)
        redefe["time"] = ttime(redefe, stime2)
        reatta["moves"] = tmoves(reatta, ia)
        redefe["moves"] = tmoves(redefe, id)
        reatta["speed"] = tper1move(reatta)
        redefe["speed"] = tper1move(redefe)
    # -----------------------------------------------------------------------------------------------
    def agent_work(Agent, board, mark, config):
        obs = Obs(board, mark)
        start = time.time()
        im = Agent(obs, config)
        t = time.time() - start
        board_next = inject(im, obs.mark, board)
        return im, t, board_next
    # -----------------------------------------------------------------------------------------------
    def write_Connect4(ims):
        #     with open("Connect4.txt", "w") as file:
        #         file.write(str(ims))
        pass 
    # -----------------------------------------------------------------------------------------------
    def write_uraConnect(ura_ims, Attack, Defense):
        #     try:
        #         with open("uraConnect.txt", "w") as file:
        #             stf = str(ura_ims) + '\n' + name(Attack) + ' - ' + name(Defense)
        #             file.write(stf)
        #     finally: return
        pass
    # -----------------------------------------------------------------------------------------------
    def name(agent): return agent.__name__.replace("my_","")
    # -----------------------------------------------------------------------------------------------

    Agents = list_of_agents_to_trace # [ag1,ag2,..,agn]

    observatory = Agents[0] if only_observation_first_agent and len(Agents) > 1 else None

    total_points = { name(agent):{"T":[],"games":[[],[]],"time":[],"moves":[],"speed":[]} for agent in Agents }

    ig, board__0 = 1, [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

    for home in Agents:
        for away in Agents:
            if home!=away and (observatory==None or observatory!=None and (observatory==home or observatory==away)):
                if ig==1:
                    print("\n")
                    print(ig-1, ":")
                    print('----------------')
                    for itp in total_points.items(): print(itp)
                    ig += 1
                stime1, stime2, imts1, imts2, ims, imt, = 0, 0, [], [], [], [[], []]
                board__1 = board__0
                # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
                if start_games_at_away_first_agent:
                    Attack, Defense = away, home
                else:
                    Attack, Defense = home, away
                # >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
                total_points_Attack, total_points_Defense = [], []
                print("--------------------------")
                print(Attack.__name__, "  vs  ", Defense.__name__)
                print("-----------------------------------")
                for i in range(1,22):
                    im1, t1, board__2  = agent_work(Attack, board__1, 1, config)
                    stime1 +=t1
                    imt[0] = str(round(t1,0))
                    ims.append(im1)
                    imts1.append(imt[0])
                    write_uraConnect (ims, Attack, Defense)
                    if win_4o_in(board__2):
                        rec(total_points, 3, 0, i, i-1, stime1, stime2)
                        print('\nwin in n win in n win in n win in n win in n win in n win in n win in n')
                        print('   1 1  1   1    1     1     ', Attack.__name__, "     1     1    1   1  1 1")
                        print('win in n win in n win in n win in n win in n win in n win in n win in n\n')
                        print_to_clip (board__2, ims, i, Attack, Defense, stime1, stime2, 3,0)
                        write_Connect4 (ims)
                        write_uraConnect (ims, Attack, Defense)
                        time.sleep(1)
                        break
                    im2, t2, board__1 = agent_work(Defense, board__2, 2, config)
                    stime2 +=t2
                    imt[1] = str(round(t2, 0))
                    ims.append(im2)
                    imts2.append(imt[1])
                    print('({0}) . {1} > {2}  {3} < {4} . ({5}) --- {6} : {7} '
                          .format(round(t1,1), name(Attack), im1, im2, name(Defense), round(t2,1), i, str(ims).replace(", ",",")))
                    write_uraConnect (ims, Attack, Defense)
                    if win_4o_in(board__1):
                        rec(total_points, 0, 3, i, i, stime1, stime2)
                        print('\nwin in n win in n win in n win in n win in n win in n win in n win in n')
                        print('   2 2  2   2    2     2     ', Defense.__name__,"     2     2    2   2  2 2")
                        print('win in n win in n win in n win in n win in n win in n win in n win in n\n')
                        print_to_clip(board__1, ims, i, Attack, Defense, stime1, stime2, 0,3)
                        write_Connect4 (ims)
                        write_uraConnect(ims, Attack, Defense)
                        time.sleep(1)
                        break
                if not win_4o_in(board__1) and not win_4o_in(board__2):
                    rec(total_points, 1, 2, i, i, stime1, stime2)
                    print('\nDRAW RAW AW W DRAW RAW AW W DRAW RAW AW W DRAW RAW AW W DRAW RAW AW W DRAW RAW AW W')
                    print('  0 0  0   0    0     ', Attack.__name__, " - ", Defense.__name__, "     0    0   0  0 0")
                    print('DRAW RAW AW W DRAW RAW AW W DRAW RAW AW W DRAW RAW AW W DRAW RAW AW W DRAW RAW AW W\n')
                    print_to_clip(board__1, ims, i, Attack, Defense, stime1, stime2, 1,2)
                    write_Connect4 (ims)
                    write_uraConnect(ims, Attack, Defense)
                    time.sleep(3)
                print("\n")
                print(ig,":\n") # , ": ", total_points,
                for itp in total_points.items():
                    print(itp)
                print("\n")
                ig +=1

## [round_robin](https://www.kaggle.com/code/dott1718/kdb-workshop)

In [13]:
def create(): 
    return make("connectx", configuration={"inarow":4,"columns":7,"rows":6})
       
def run_game(agent1, agent2, shuffle=False):
    env = create()
    swap = shuffle and random.choice([True, False])
    if swap:
        env.run([agent2, agent1])
    else:
        env.run([agent1, agent2])

    if env.steps[-1][0].status == "TIMEOUT":
        rew1 = 0
        rew2 = 1
    elif env.steps[-1][1].status == "TIMEOUT":
        rew2 = 0
        rew1 = 1
    else:
        rew1 = (env.steps[-1][0].reward + 1.0) / 2
        rew2 = (env.steps[-1][1].reward + 1.0) / 2

    time1 = max(0, env.steps[-1][0].observation.remainingOverageTime)
    time2 = max(0, env.steps[-1][1].observation.remainingOverageTime)

    if swap:
        return rew2, rew1, time2, time1, len(env.steps)
    else:
        return rew1, rew2, time1, time2, len(env.steps)


def run_games(agent1, agent2, n_games=1, n_jobs=-1, shuffle=True):
    if n_jobs != 1:
        game_res = Parallel(
            n_jobs=-1, temp_folder="/tmp", max_nbytes=None, backend="loky"
        )(delayed(run_game)(agent1, agent2, shuffle) for _ in range(n_games))
    else:
        game_res = [run_game(agent1, agent2, shuffle) for _ in range(n_games)]

    return [np.mean([x[i] for x in game_res]) for i in range(5)]


def round_robin(agents, n_games=1, n_jobs=-1):
    res = {
        "name": [],
        "win_rate": [],
        "rem_time": [],
    }
    for i in range(len(agents)):
        for j in range(len(agents)):
            if j <= i:
                continue
            game_res = run_games(agents[i], agents[j], n_games, n_jobs)
            res["name"].append(agents[i] if type(agents[i]) == str else agents[i].__name__)
            res["win_rate"].append(game_res[0])
            res["rem_time"].append(game_res[2])
            res["name"].append(agents[j] if type(agents[j]) == str else agents[j].__name__)
            res["win_rate"].append(game_res[1])
            res["rem_time"].append(game_res[3])
    return (
        pd.DataFrame(res)
        .groupby("name")
        .mean()
        .reset_index()
        .sort_values(["win_rate", "rem_time"], ascending=False)
        .reset_index(drop=True)
    )

In [14]:
import inspect
import os

def write_agent_to_file(function, file):
    with open(file, "a" if os.path.exists(file) else "w") as f:
        f.write(inspect.getsource(function))
        print(function, "written to", file)

write_agent_to_file(ag_minimax_20xfaster,"agent_minimax_20xfaster.py")
write_agent_to_file(ag_heuristic5xfaster,"agent_heuristic5xfaster.py")
write_agent_to_file(ag_KLAlexisCook_abnj,"agent_KLAlexisCook_abnj.py")
# write_agent_to_file(ag__faster__CYTHON__,"submission_ag__Faster__CYTHON__.py")

<function ag_minimax_20xfaster at 0x7cf16a432680> written to agent_minimax_20xfaster.py
<function ag_heuristic5xfaster at 0x7cf16a432710> written to agent_heuristic5xfaster.py
<function ag_KLAlexisCook_abnj at 0x7cf1622c9870> written to agent_KLAlexisCook_abnj.py


In [15]:
debugger_c4([
    ag_minimax_20xfaster, 
    ag_heuristic5xfaster, 
    ag_KLAlexisCook_abnj, 
    ag__faster__CYTHON__,
])



0 :
----------------
('ag_minimax_20xfaster', {'T': [], 'games': [[], []], 'time': [], 'moves': [], 'speed': []})
('ag_heuristic5xfaster', {'T': [], 'games': [[], []], 'time': [], 'moves': [], 'speed': []})
('ag_KLAlexisCook_abnj', {'T': [], 'games': [[], []], 'time': [], 'moves': [], 'speed': []})
('ag__faster__CYTHON__', {'T': [], 'games': [[], []], 'time': [], 'moves': [], 'speed': []})
--------------------------
ag_minimax_20xfaster   vs   ag_heuristic5xfaster
-----------------------------------
(0.0) . ag_minimax_20xfaster > 3  3 < ag_heuristic5xfaster . (0.0) --- 1 : [3,3] 
(0.0) . ag_minimax_20xfaster > 3  3 < ag_heuristic5xfaster . (0.0) --- 2 : [3,3,3,3] 
(0.0) . ag_minimax_20xfaster > 1  4 < ag_heuristic5xfaster . (2.3) --- 3 : [3,3,3,3,1,4] 
(4.9) . ag_minimax_20xfaster > 1  1 < ag_heuristic5xfaster . (2.3) --- 4 : [3,3,3,3,1,4,1,1] 
(4.8) . ag_minimax_20xfaster > 2  1 < ag_heuristic5xfaster . (1.5) --- 5 : [3,3,3,3,1,4,1,1,2,1] 

win in n win in n win in n win in n win in

## bot battle

In [16]:
# round_robin(agents=[
#     ag_minimax_20xfaster, 
#     ag_heuristic5xfaster, 
#     ag_KLAlexisCook_abnj, 
#     ag__faster__CYTHON__,
# ], n_games=2)